# About

This notebook extracts spectral and date features of NAIP images at random points within polygons in the 'polygons_form_naip_images' folder. These polygons are known ice plant and non ice plant locations within a specific NAIP image. 


Once the area of interest and years are specified, the notebook samples first polygons labeled as ice plant locations and then polygons labeled as non-ice plant locations. Two methods for sampling polygons are implemented in the `extracting_points_from_polygons` module, and both are used in this notebook. The first one samples a fixed fraction of the pixels in each polygon. The second one samples a fixed fraction of the pixels in each polygon up to a maximum number of points. Polygons vary significantly in size, so simply sampling a fraction of the pixels in each polygon would result in an over-sampling of bigger polygons (most often those corresponding to non-ice plant locations), which in turn would greatly unbalance the training set towards one label. The parameters used in this notebook were determined to obtain a final training set with a XXX proportion of non-ice plant to ice plant points. 

**NOTEBOOK VARIABLES:**

- `aois` (array): These are the areas of interest where we collected the polygons we want to sample. Must be a subset of: `['campus_lagoon','carpinteria','gaviota','point_conception']`. 

- `years` (array): can be any subset of `[2012, 2014, 2016, 2018, 2020]`. If aoi = 'point_conception', then 2016 will not be included in the outcome since there are no NAIP images to sample from that year. 

- `sample_fraction` (float in (0,1]): fraction of points to sample from each polygon

- `max_sample` (int): maximum number of points to sample from a polygon

- `verbose` (bool): whether to print the stats of how many points were sampled per year and area of interest as the notebook runs

- `write_stats` (bool): whether to save as a csv file the stats of how many points were sampled from each year and area of interest

**OUTPUT:**

The output is a data frame of points with the following features:

- x, y: coordinates of point *p* 
- pts_crs: CRS of coordinates x, y
- naip_id: itemid of the NAIP from which *p* was sampled from
- polygon_id: id of the polygon from which *p* was sampled from
- iceplant: whether point *p* corresponds to a confirmed iceplant location or a confirmed non-iceplant location (0 = non-iceplant, 1 = iceplant)
- r, g, b, nir: Red, Green, Blue, and NIR values of NAIP scene with naip_id at coordinates of point *p*
- ndvi: computed for each point using the Red and NIR bands
- year, month, day_in_year: year, month, and day of the year when the NAIP image was collected
- aoi: name of the area of interest where the points were sampled from


The data frames are saved in the 'temp' folder as a csv file. Filenames have the structure: `aoi_points_year.csv'`
The stats are saved in the current working directory as `stats_1_sampling_pts_from_polygons.csv`.

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import os
import geopandas as gpd

import sample_rasters as sr

# Specify notebook variables

In [2]:
# ***************************************************
# ************* NOTEBOOK VARIABLES ******************

aois = ['campus_lagoon','carpinteria','gaviota','point_conception']

# years = array of years, can be any subset from [2012, 2014, 2016, 2018, 2020]
years = [2012, 2014, 2016, 2018, 2020]

# sample 90% of pts in each polygon
sample_fraction = 0.9

# maximum number of pts to sample in a polygon
max_sample = 500

# print stats as notebook runs
verbose = True

# save stats
write_stats = True

# ***************************************************
# ***************************************************

# Sample points

In [3]:
# initialize sampling statistcs df
if write_stats:
    stats = []

# sample points
for aoi in aois:
    for year in years:
        
        if ('point_conception' != aoi) or (year != 2016):  #there's no data for Point Conception on 2016
            
            # open polygons
            fp = sr.path_to_polygons(aoi,year)
            polys = gpd.read_file(fp)
            # -------------------------
            # select iceplant polygons and sample sample_fraction of pts in each polygon 
            polys_ice = polys.loc[polys.iceplant==1]
            polys_ice.reset_index(inplace=True, drop=True)
            
            pts_ice = sr.sample_naip_from_polys_no_warnings(polys = polys_ice,
                                                            class_name = 'iceplant',
                                                            itemid = polys.naip_id[0], 
                                                            param = 'fraction',
                                                            sample_fraction = sample_fraction)  
            # -------------------------
            # select non-iceplant polygons and sample sample_fraction of pts in each polygon,  but at most max_sample points 
            polys_nonice = polys.loc[polys.iceplant==0]
            polys_nonice.reset_index(inplace=True, drop=True)
            
            pts_nonice = sr.sample_naip_from_polys_no_warnings(polys = polys_nonice,
                                                            class_name = 'iceplant',
                                                            itemid = polys.naip_id[0], 
                                                            param = 'sliding',
                                                            sample_fraction = sample_fraction,
                                                            max_sample = max_sample)  
            # -------------------------            
            # assemble into single dataframe
            pts = pd.concat([pts_ice,pts_nonice])
            pts['aoi'] = aoi # add name of aoi
            pts['ndvi']=(pts.nir.astype('int16') - pts.r.astype('int16'))/(pts.nir.astype('int16') + pts.r.astype('int16'))# add ndvi as feature

            # -------------------------           
            # save points as csv in temp folder            
            tmp_path = os.path.join(os.getcwd(),'temp')  # create temp directory if needed
            if not os.path.exists(tmp_path):
                os.makedirs(tmp_path)
            
            # change to function
            fp = sr.path_to_spectral_pts(aoi, year)
            pts.to_csv(fp, index=False)
            
            # -------------------------
            # print sample statistics
            if verbose == True:
                print('************ '+aoi+ ' ' +str(year)+' ************')
                sr.iceplant_proportions(pts.iceplant)
                print( '---------------------------------------')
                
            # -------------------------
            # keep track of statistics for saving
            if write_stats == True:
                n_ice =  pts_ice.shape[0]
                n_nonice =  pts_nonice.shape[0]
                total = n_ice + n_nonice
                
                stat = [aoi, 
                     year, 
                     str(round(n_nonice/n_ice,1))+':1', 
                     round(n_ice/total*100,2),
                     round(n_nonice/total*100,2),
                     n_ice,
                     n_nonice
                    ]
                stats.append(stat)

************ campus_lagoon 2012 ************
no-iceplant:iceplant ratio     1.5 :1
          counts  percentage
iceplant                    
0          14012       60.06
1           9317       39.94

---------------------------------------
************ campus_lagoon 2014 ************
no-iceplant:iceplant ratio     1.6 :1
          counts  percentage
iceplant                    
0          10529       61.72
1           6529       38.28

---------------------------------------
************ campus_lagoon 2016 ************
no-iceplant:iceplant ratio     0.8 :1
          counts  percentage
iceplant                    
0.0        10959       43.15
1.0        14439       56.85

---------------------------------------
************ campus_lagoon 2018 ************
no-iceplant:iceplant ratio     1.4 :1
          counts  percentage
iceplant                    
0          29844       58.95
1          20782       41.05

---------------------------------------
************ campus_lagoon 2020 ********

In [4]:
if write_stats:     
    stats_df = pd.DataFrame(stats, 
                            columns=['aoi', 'year', 'ratio','perc_ice','perc_nonice','n_ice','n_nonice'])

    # save points as csv in temp folder
    fp = os.path.join(os.getcwd(),  
                      'stats_sampling_pts_from_polygons.csv')
    stats_df.to_csv(fp, index=False)